## Predictor Algorythm

In [1]:
import numpy as np
import random
random.seed(1)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM
from htm.bindings.algorithms import Predictor

%matplotlib inline

In [2]:
tm = TM(
  columnDimensions = (2048,),
  cellsPerColumn=8,
  initialPermanence=0.21,
  connectedPermanence=0.3,
  minThreshold=15,
  maxNewSynapseCount=40,
  permanenceIncrement=0.1,
  permanenceDecrement=0.1,
  activationThreshold=15,
  predictedSegmentDecrement=0.01,
  )

In [12]:
sparsity   = 0.02
sparseCols = int(tm.numberOfColumns() * sparsity)
dataset    = {inp : SDR( tm.numberOfColumns() ) for inp in "ABCDXY"}
for i, inp in enumerate("ABCDXY"):
  dataset[inp].dense[ i * sparseCols : (i + 1) * sparseCols ] = 1
  dataset[inp].dense = dataset[inp].dense # この行は、SDR の高密度データが変更されたことを SDR に通知する.
  #print("Input", inp, "is bits at indices: [",  i * sparseCols, '-', (i + 1) * sparseCols, ']')
  print(dataset[inp])
print(sorted(dataset.keys()))
labels=[i for i in range(len(dataset))]
print(labels)

seq1 = "ABCD"
seq2 = "XBCY"
seqT = "ABCDXY"

SDR( 2048 ) 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39
SDR( 2048 ) 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79
SDR( 2048 ) 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119
SDR( 2048 ) 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159
SDR( 2048 ) 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199
SDR( 2048 ) 200, 201, 202, 203, 204, 205, 206, 207, 208

In [8]:
for t in range(1):
  seq = random.choice([ seq1, seq2 ])
  for inp in seq:
    tm.compute( dataset[inp], learn=True)
    print(tm.getActiveCells() )


SDR( 2048, 8 ) 1282, 1285, 1290, 1293, 1298, 1301, 1306, 1309, 1314, 1317, 1322, 1325, 1330, 1333, 1338, 1341, 1346, 1349, 1354, 1357, 1362, 1365, 1370, 1373, 1378, 1381, 1386, 1389, 1394, 1397, 1402, 1405, 1410, 1413, 1418, 1421, 1426, 1429, 1434, 1437, 1442, 1445, 1450, 1453, 1458, 1461, 1466, 1469, 1474, 1477, 1482, 1485, 1490, 1493, 1498, 1501, 1506, 1509, 1514, 1517, 1522, 1525, 1530, 1533, 1538, 1541, 1546, 1549, 1554, 1557, 1562, 1565, 1570, 1573, 1578, 1581, 1586, 1589, 1594, 1597
SDR( 2048, 8 ) 321, 325, 329, 333, 337, 341, 345, 349, 353, 357, 361, 365, 369, 373, 377, 381, 385, 389, 393, 397, 401, 405, 409, 413, 417, 421, 425, 429, 433, 437, 441, 445, 449, 453, 457, 461, 465, 469, 473, 477, 481, 485, 489, 493, 497, 501, 505, 509, 513, 517, 521, 525, 529, 533, 537, 541, 545, 549, 553, 557, 561, 565, 569, 573, 577, 581, 585, 589, 593, 597, 601, 605, 609, 613, 617, 621, 625, 629, 633, 637
SDR( 2048, 8 ) 641, 643, 649, 651, 657, 659, 665, 667, 673, 675, 681, 683, 689, 691, 697, 69